In [1]:
# Import necessary modules
import pandas as pd
import numpy as np

In [2]:
# Upload data from csv's
df_2015 = pd.read_csv('..\Data\pitches_2015.csv', index_col = 'game_id')
df_2016 = pd.read_csv('..\Data\pitches_2016.csv', index_col = 'game_id')
df_2017 = pd.read_csv('..\Data\pitches_2017.csv', index_col = 'game_id')

# Combine all seasonal dataframes into one
df = pd.concat([df_2015, df_2016, df_2017])

# Drop 'Unnamed' column
df.drop('Unnamed: 0', axis = 1, inplace = True)

Each of the csv's above contain every regular season game from their respective seasons including All-Star Games. The All-Star Game splits the best players in the league into two teams and the outcome of the game has no impact on the regular season. The game also isn't played like a regular season game. Players could get swapped out every inning regardless of fatigue or performance and for those reasons, I'll be removing these games. The corresponding game_id for the All-Star games are below.

In [3]:
all_star_gameid = [414988, 448202, 491434]

for i in all_star_gameid:
    df.drop(i, inplace=True)
df.reset_index(inplace=True)

# View head of df
df.head()

,game_id,batSide_code,batSide_des,batter,batter_id,call,call_des,count_balls,count_strikes,inning_num,inning_top_bot,pitchHand_code,pitchHand_des,pitch_locx,pitch_locy,pitch_speed,pitch_type,pitch_type_des,pitcher,pitcher_id
0,413661,L,Left,Matt Carpenter,572761,C,Called Strike,0,0,1,top,L,Left,0.416,2.963,92.9,FF,Four-Seam Fastball,Jon Lester,452657
1,413661,L,Left,Matt Carpenter,572761,S,Swinging Strike,0,1,1,top,L,Left,-0.191,2.347,92.8,FF,Four-Seam Fastball,Jon Lester,452657
2,413661,L,Left,Matt Carpenter,572761,F,Foul,0,2,1,top,L,Left,-0.518,3.284,94.1,FF,Four-Seam Fastball,Jon Lester,452657
3,413661,L,Left,Matt Carpenter,572761,B,Ball,0,2,1,top,L,Left,-0.641,1.221,91.0,FF,Four-Seam Fastball,Jon Lester,452657
4,413661,L,Left,Matt Carpenter,572761,B,Ball,1,2,1,top,L,Left,-1.821,2.083,75.4,CU,Curveball,Jon Lester,452657


In [4]:
df.describe()

,game_id,batter_id,count_balls,count_strikes,inning_num,pitch_locx,pitch_locy,pitch_speed,pitcher_id
count,2.134073e+06,2.134073e+06,2.134073e+06,2.134073e+06,2.134073e+06,2.134073e+06,2.134073e+06,2.129466e+06,2.134073e+06
mean,4.518092e+05,5.099545e+05,8.796077e-01,8.800168e-01,4.997819e+00,2.329170e-01,2.589080e+00,8.830204e+01,5.182730e+05
std,3.129214e+04,8.181368e+04,9.651697e-01,8.248188e-01,2.664939e+00,5.419492e+00,7.331222e+00,6.048753e+00,7.672827e+04
min,4.136610e+05,1.125260e+05,0.000000e+00,0.000000e+00,1.000000e+00,-1.054333e+01,-5.183664e+00,3.390000e+01,1.125260e+05
25%,4.155160e+05,4.559760e+05,0.000000e+00,0.000000e+00,3.000000e+00,-6.010000e-01,1.656000e+00,8.430000e+01,4.585840e+05
50%,4.481210e+05,5.185950e+05,1.000000e+00,1.000000e+00,5.000000e+00,6.000000e-03,2.263467e+00,8.970000e+01,5.191410e+05
75%,4.907290e+05,5.721140e+05,1.000000e+00,2.000000e+00,7.000000e+00,6.064463e-01,2.868000e+00,9.290000e+01,5.807920e+05
max,4.925280e+05,6.697200e+05,4.000000e+00,2.000000e+00,1.900000e+01,2.472733e+02,2.472400e+02,1.050000e+02,6.647010e+05


In columns ``call`` and ``call_des``, there are a multitude of values that these columns can take. Each code in the ``call`` columns should correspond to a description as to what happened during the play. It'll be useful to understand what each of the codes mean.

In [5]:
# Determine what the call codes mean by sorting by code and description
call_codes = df.dropna().groupby('call')['call_des'].unique()
print(call_codes)

call
*B                 [Ball In Dirt]
B                          [Ball]
C                 [Called Strike]
D               [In play, no out]
E               [In play, run(s)]
F     [Foul, Foul (Runner Going)]
I                   [Intent Ball]
L                     [Foul Bunt]
M                   [Missed Bunt]
P                      [Pitchout]
Q             [Swinging Pitchout]
R                 [Foul Pitchout]
S               [Swinging Strike]
T                      [Foul Tip]
W     [Swinging Strike (Blocked)]
X               [In play, out(s)]
Z               [In play, run(s)]
Name: call_des, dtype: object


As printed in the above output, there are multiple codes that could mean the same thing in regards to the outcome of a pitch. For example, ``Ball in Dirt`` and ``Intent Ball`` could simply mean ``Ball``. The codes could therefore be simplified to mean the same things. However ``W``(``Swinging Strike (Blocked)``) poses an interesting problem. When a batter swings at a ball in the dirt and misses with 2 strikes, the batter as the option to run to first base. If he successfully makes it to first, he's considered safe. It counts as a a strikeout for the pitcher but not an official out during the game. To help simplify the codes, consider how many times ``W`` occured with 2 strikes and standardizing the outcome to strikes if it'll have minimal effect.

In [6]:
# Count how many 'W' with 2 strikes
swinging_strike_blocked = df[(df['call'] == 'W') & (df['count_strikes'] == 2)]
print(len(swinging_strike_blocked))

13015


In [7]:
percent_ssb = len(swinging_strike_blocked)/len(df_2015)
percent_ssb_outs = len(swinging_strike_blocked)/((2418+2424+2429)*27*2)

Seeing as blocked swinging strikes accounts for 1.9% of pitches and ~3% of outs in those 3 seasons, I'll assume no errors occured on such plays and categorize the pitch as a swinging strike and if the batter had 2 strikes, they'd be 'out'. Next is to create a dictionary to update all the existing ``call``.

In [8]:
# Create a dictionary that will be used to update the 'call' codes
new_call_codes_dict = {'*B':'B',
                       'B':'B',
                       'C':'C',
                       'D':'H',
                       'E':'H',
                       'F':'F',
                       'I':'B',
                       'L':'S',
                       'M':'S',
                       'P':'B',
                       'Q':'S',
                       'R':'F',
                       'S':'S',
                       'T':'S',
                       'W':'S',
                       'X':'X',
                       'Z':'H'}

In [9]:
# Simplify 'call' using 'new_call_codes_dict'
df['call'].replace(new_call_codes_dict, inplace=True)

Now that the ``call`` column has been simplified, we can use ``call_des`` to fill in the ``nan`` in the ``call`` column. This is done by duplicating the ``call_des`` column and using ``.replace`` to change them to ``call_code``.

In [10]:
# Create new dictionary to fill 'nan' in 'call' based on 'call_des'
des_codes = df.dropna().groupby('call_des')['call'].unique().to_dict()
des_codes = {i:des_codes[i][0] for i in des_codes.keys()}
des_codes

{'Ball': 'B',
 'Ball In Dirt': 'B',
 'Called Strike': 'C',
 'Foul': 'F',
 'Foul (Runner Going)': 'F',
 'Foul Bunt': 'S',
 'Foul Pitchout': 'F',
 'Foul Tip': 'S',
 'In play, no out': 'H',
 'In play, out(s)': 'X',
 'In play, run(s)': 'H',
 'Intent Ball': 'B',
 'Missed Bunt': 'S',
 'Pitchout': 'B',
 'Swinging Pitchout': 'S',
 'Swinging Strike': 'S',
 'Swinging Strike (Blocked)': 'S'}

In [11]:
# Update 'call' to fill in 'nan'
df['call_codes'] = df['call_des']
df['call_codes'].replace(des_codes, inplace=True)
df.drop('call', axis=1, inplace=True)
df.head()

,game_id,batSide_code,batSide_des,batter,batter_id,call_des,count_balls,count_strikes,inning_num,inning_top_bot,pitchHand_code,pitchHand_des,pitch_locx,pitch_locy,pitch_speed,pitch_type,pitch_type_des,pitcher,pitcher_id,call_codes
0,413661,L,Left,Matt Carpenter,572761,Called Strike,0,0,1,top,L,Left,0.416,2.963,92.9,FF,Four-Seam Fastball,Jon Lester,452657,C
1,413661,L,Left,Matt Carpenter,572761,Swinging Strike,0,1,1,top,L,Left,-0.191,2.347,92.8,FF,Four-Seam Fastball,Jon Lester,452657,S
2,413661,L,Left,Matt Carpenter,572761,Foul,0,2,1,top,L,Left,-0.518,3.284,94.1,FF,Four-Seam Fastball,Jon Lester,452657,F
3,413661,L,Left,Matt Carpenter,572761,Ball,0,2,1,top,L,Left,-0.641,1.221,91.0,FF,Four-Seam Fastball,Jon Lester,452657,B
4,413661,L,Left,Matt Carpenter,572761,Ball,1,2,1,top,L,Left,-1.821,2.083,75.4,CU,Curveball,Jon Lester,452657,B


In [12]:
# Verify 'call_codes' doesn't contain 'nan'
df.isna().sum()

game_id              0
batSide_code         0
batSide_des          0
batter               0
batter_id            0
call_des             0
count_balls          0
count_strikes        0
inning_num           0
inning_top_bot       0
pitchHand_code       0
pitchHand_des        0
pitch_locx           0
pitch_locy           0
pitch_speed       4607
pitch_type        4670
pitch_type_des    4670
pitcher              0
pitcher_id           0
call_codes           0
dtype: int64

There are a number of ``nan`` in the columns ``pitch_speed``, ``pitch_type``, and ``pitch_type_des``. It may be impossible to fill these values in without manually checking what they are so dropping them is necessary and is acceptable since there's +4,600 out of +2,000,000 observations.

In [13]:
# Remove observations with 'nan'
df.dropna(inplace=True)

In [14]:
df.describe()

,game_id,batter_id,count_balls,count_strikes,inning_num,pitch_locx,pitch_locy,pitch_speed,pitcher_id
count,2.129403e+06,2.129403e+06,2.129403e+06,2.129403e+06,2.129403e+06,2.129403e+06,2.129403e+06,2.129403e+06,2.129403e+06
mean,4.517838e+05,5.099469e+05,8.795573e-01,8.801612e-01,4.997960e+00,-3.245301e-03,2.257968e+00,8.830206e+01,5.182664e+05
std,3.128785e+04,8.180790e+04,9.650989e-01,8.247927e-01,2.664643e+00,9.018426e-01,9.459449e-01,6.048719e+00,7.672944e+04
min,4.136610e+05,1.125260e+05,0.000000e+00,0.000000e+00,1.000000e+00,-1.054333e+01,-5.183664e+00,3.390000e+01,1.125260e+05
25%,4.155140e+05,4.559760e+05,0.000000e+00,0.000000e+00,3.000000e+00,-6.029142e-01,1.654000e+00,8.430000e+01,4.585370e+05
50%,4.481180e+05,5.185950e+05,1.000000e+00,1.000000e+00,5.000000e+00,3.835833e-03,2.261000e+00,8.970000e+01,5.191410e+05
75%,4.907280e+05,5.721140e+05,1.000000e+00,2.000000e+00,7.000000e+00,6.020000e-01,2.863000e+00,9.290000e+01,5.807920e+05
max,4.925280e+05,6.697200e+05,4.000000e+00,2.000000e+00,1.900000e+01,1.295291e+01,1.248854e+01,1.050000e+02,6.647010e+05


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2129403 entries, 0 to 2134072
Data columns (total 20 columns):
game_id           int64
batSide_code      object
batSide_des       object
batter            object
batter_id         int64
call_des          object
count_balls       int64
count_strikes     int64
inning_num        int64
inning_top_bot    object
pitchHand_code    object
pitchHand_des     object
pitch_locx        float64
pitch_locy        float64
pitch_speed       float64
pitch_type        object
pitch_type_des    object
pitcher           object
pitcher_id        int64
call_codes        object
dtypes: float64(3), int64(6), object(11)
memory usage: 341.2+ MB


Now that ``call_code`` is standardized and simplified, an ``is_out`` column could be added. Of course, if ``call_code`` is ``X``, ``is_out`` should be ``True`` or ``1``. The other way ``is_out`` is ``True`` is if a batter gets called out on strikes or swings and misses with 2 strikes.

In [16]:
# Create new column 'is_out' to categorize pitches that resulted in outs
df['is_out'] = [int(df.call_codes[i] == 'X') for i in df.index]

In [17]:
df.head(10)

,game_id,batSide_code,batSide_des,batter,batter_id,call_des,count_balls,count_strikes,inning_num,inning_top_bot,...,pitchHand_des,pitch_locx,pitch_locy,pitch_speed,pitch_type,pitch_type_des,pitcher,pitcher_id,call_codes,is_out
0,413661,L,Left,Matt Carpenter,572761,Called Strike,0,0,1,top,...,Left,0.416,2.963,92.9,FF,Four-Seam Fastball,Jon Lester,452657,C,0
1,413661,L,Left,Matt Carpenter,572761,Swinging Strike,0,1,1,top,...,Left,-0.191,2.347,92.8,FF,Four-Seam Fastball,Jon Lester,452657,S,0
2,413661,L,Left,Matt Carpenter,572761,Foul,0,2,1,top,...,Left,-0.518,3.284,94.1,FF,Four-Seam Fastball,Jon Lester,452657,F,0
3,413661,L,Left,Matt Carpenter,572761,Ball,0,2,1,top,...,Left,-0.641,1.221,91.0,FF,Four-Seam Fastball,Jon Lester,452657,B,0
4,413661,L,Left,Matt Carpenter,572761,Ball,1,2,1,top,...,Left,-1.821,2.083,75.4,CU,Curveball,Jon Lester,452657,B,0
5,413661,L,Left,Matt Carpenter,572761,"In play, out(s)",2,2,1,top,...,Left,0.627,2.397,92.9,FF,Four-Seam Fastball,Jon Lester,452657,X,1
6,413661,L,Left,Jason Heyward,518792,Ball,0,0,1,top,...,Left,-1.088,1.610,93.3,FF,Four-Seam Fastball,Jon Lester,452657,B,0
7,413661,L,Left,Jason Heyward,518792,"In play, no out",1,0,1,top,...,Left,-0.257,2.047,89.3,FC,Cutter,Jon Lester,452657,H,0
8,413661,R,Right,Matt Holliday,407812,Ball,0,0,1,top,...,Left,1.470,2.350,92.1,FF,Four-Seam Fastball,Jon Lester,452657,B,0
9,413661,R,Right,Matt Holliday,407812,Ball,1,0,1,top,...,Left,-1.337,1.898,89.3,FF,Four-Seam Fastball,Jon Lester,452657,B,0


In [18]:
# Update 'is_out' to include strikeouts as 1
df.loc[((df['call_codes'] == 'S')|(df['call_codes'] == 'C')) & (df['count_strikes'] == 2), 'is_out'] = 1

Now that the outcomes of the pitches are assigned to ``is_out`` as a numeric value, the next step is to turn the other variables to numeric values. For the ``batSide_code`` and ``pitchHand_code``, new columns ``batSide_isRight`` and ``pitchHand_isRight`` will added to take on values of ``0`` for left-handed and ``1`` for right-handed.

In [19]:
df['pitchHand_isRight'] = (df['pitchHand_code'] == 'R').astype(int)
df['batSide_isRight'] = (df['batSide_code'] == 'R').astype(int)

In [20]:
df.tail()

,game_id,batSide_code,batSide_des,batter,batter_id,call_des,count_balls,count_strikes,inning_num,inning_top_bot,...,pitch_locy,pitch_speed,pitch_type,pitch_type_des,pitcher,pitcher_id,call_codes,is_out,pitchHand_isRight,batSide_isRight
2134068,492528,L,Left,Greg Bird,595885,Ball,0,1,9,bottom,...,0.873194,95.9,FT,Two-Seam Fastball,Roberto Osuna,532077,B,0,1,0
2134069,492528,L,Left,Greg Bird,595885,Ball,1,1,9,bottom,...,0.800587,82.8,CH,Changeup,Roberto Osuna,532077,B,0,1,0
2134070,492528,L,Left,Greg Bird,595885,Ball,2,1,9,bottom,...,2.756071,94.7,FF,Four-Seam Fastball,Roberto Osuna,532077,B,0,1,0
2134071,492528,L,Left,Greg Bird,595885,Foul,3,1,9,bottom,...,2.876476,91.8,FC,Cutter,Roberto Osuna,532077,F,0,1,0
2134072,492528,L,Left,Greg Bird,595885,"In play, out(s)",3,2,9,bottom,...,3.248982,91.9,FC,Cutter,Roberto Osuna,532077,X,1,1,0


To categorize ``pitch_type``, pandas ``get_dummies`` function could be used to enumerate the column.

In [21]:
# Use 'get_dummies' on 'pitch_type_des'
df = pd.get_dummies(df, prefix=['pitch_type', 'call'], columns = ['pitch_type_des', 'call_codes'])
df.drop('call_X', inplace=True, axis=1)

It may not be used in the first get of models, but the outcome of the pitch, ``call_codes``, may also be used as a target variable instead of only having ``is_out``.

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2129403 entries, 0 to 2134072
Data columns (total 43 columns):
game_id                          int64
batSide_code                     object
batSide_des                      object
batter                           object
batter_id                        int64
call_des                         object
count_balls                      int64
count_strikes                    int64
inning_num                       int64
inning_top_bot                   object
pitchHand_code                   object
pitchHand_des                    object
pitch_locx                       float64
pitch_locy                       float64
pitch_speed                      float64
pitch_type                       object
pitcher                          object
pitcher_id                       int64
is_out                           int64
pitchHand_isRight                int32
batSide_isRight                  int32
pitch_type_Changeup              uint8
pitch_type_Cur

Next would be to reorder the columns so that predictor variables are grouped consecutively next to target variables.

In [23]:
# Get column names as a list
cols = df.columns.tolist()

In [24]:
# Reorder the column names
detail_cols = cols[:6] + cols[9:12] + cols[15:18]
predictor_cols = cols[6:9] + cols[12:15] + cols[19:38]
target_cols = cols[38:] + cols[18:19]

reorder_cols = detail_cols + predictor_cols + target_cols
df = df[reorder_cols]

In [25]:
# Export df to csv
df.to_csv('..\\Data\\all_pitches.csv', index=False)

In [26]:
detail_cols

['game_id',
 'batSide_code',
 'batSide_des',
 'batter',
 'batter_id',
 'call_des',
 'inning_top_bot',
 'pitchHand_code',
 'pitchHand_des',
 'pitch_type',
 'pitcher',
 'pitcher_id']

In [27]:
predictor_cols

['count_balls',
 'count_strikes',
 'inning_num',
 'pitch_locx',
 'pitch_locy',
 'pitch_speed',
 'pitchHand_isRight',
 'batSide_isRight',
 'pitch_type_Changeup',
 'pitch_type_Curveball',
 'pitch_type_Cutter',
 'pitch_type_Eephus',
 'pitch_type_Fastball',
 'pitch_type_Forkball',
 'pitch_type_Four-Seam Fastball',
 'pitch_type_Int. Ball',
 'pitch_type_Knuckle Curve',
 'pitch_type_Knuckleball',
 'pitch_type_Pitchout',
 'pitch_type_Screwball',
 'pitch_type_Sinker',
 'pitch_type_Slider',
 'pitch_type_Splitter',
 'pitch_type_Two-Seam Fastball',
 'pitch_type_Unknown']

In [28]:
target_cols

['call_B', 'call_C', 'call_F', 'call_H', 'call_S', 'is_out']